In [1]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import division

import glob
import numpy as np
from PIL import Image
from mpl_toolkits.mplot3d import Axes3D

In [5]:
from compactness.helpers import getSphere
from compactness.helpers import rotate3D

from compactness.moments import immoment3D

from compactness.cubeness import __int_max_abs__
from compactness.cubeness import cubeness_mz

In [7]:
# Move to helpers ?
def plotVolume(volume):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    X, Y, Z = volume.nonzero()
    ax.plot(X, Y, Z, '.')
#plotVolume(volume)

In [48]:
# Test - Display Cube
plotVolume(np.ones((140,140,140)))

In [8]:
# Test - Display sphere
volume = getSphere(50)
plotVolume(volume)

In [23]:
# Test rotate3D
nSamples = 10
X = np.zeros(nSamples)
Y = np.linspace(-1,1,nSamples)
Z = np.zeros(nSamples)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for rx in np.linspace(0,pi, 5):
    for ry in np.linspace(0,pi, 5):
        X_,Y_,Z_ = rotate3D(X,Y,Z, rx, ry)
        ax.plot(X_, Y_, Z_,'+')


In [75]:
# Test immmoments
volume = getSphere(50)
X,Y,Z = volume.nonzero()

# Moment 0,0,0 is the volume of the object
assert volume.sum()==immoment3D(X,Y,Z,0,0,0)
assert len(X)==immoment3D(X,Y,Z,0,0,0)

m000 = immoment3D(X,Y,Z,0,0,0)
m100 = immoment3D(X,Y,Z,1,0,0)
m010 = immoment3D(X,Y,Z,0,1,0)
m001 = immoment3D(X,Y,Z,0,0,1)
print m000
print m100
print m010
print m001

print m100 / m000
print m010 / m000
print m001 / m000

    # Centering 
    X = X-(m100 / m000)
    Y = Y-(m010 / m000)
    Z = Z-(m001 / m000)

    m000 = immoment3D(X,Y,Z,0,0,0)
    m100 = immoment3D(X,Y,Z,1,0,0)
    m010 = immoment3D(X,Y,Z,0,1,0)
    m001 = immoment3D(X,Y,Z,0,0,1)
print '=============='
print m000
print m100
print m010
print m001

710403
47806861
49495500
56286828
67.2954097885
69.6724253698
79.2322498638
710403.0
-2.77516392089e-08
3.87539955682e-09
-1.4656222902e-09


In [45]:
# Test cubeness of sphere
volume = getSphere(50)
X,Y,Z = volume.nonzero()
c,fxy = cubeness_mz(X,Y,Z)
print c

In [61]:
# Test cubeness of cube
volume = np.ones((50,50,50))
X,Y,Z = volume.nonzero()
c,fxy = cubeness_mz(X,Y,Z)
print c

1.00053361793


In [63]:
# Test -- plot fxy
angles = np.linspace(0,np.pi,fxy.shape[0])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
a,b = np.meshgrid(angles, angles)
ax.plot(a.flatten(), b.flatten(), fxy.flatten(), '+')

# And now for the problem of Manuel

In [64]:
def loadVolume(dirPath):
    files = glob.glob(dirPath + '/*.tif')
    volume = np.zeros((140, 140, 140))
    for i,filename in enumerate(files):
        img = Image.open(filename)
        # print filename
        pixels = np.asarray(img)
        volume[i,:,:] = pixels
    volume = volume > 128
    return volume.astype(np.int)

volume = loadVolume('01_xz')

In [67]:
# plotVolume(volume)

In [69]:
X,Y,Z = volume.nonzero()
c,fxy = cubeness_mz(X,Y,Z)
print c


0.629101969423
